# OpenAI API Basics

Learn the fundamentals of working with OpenAI's API.

## Setup

**Important:** You need an OpenAI API key to run this notebook.

1. Get your API key from https://platform.openai.com/api-keys
2. Set it in the cell below
3. **Never commit API keys to git!**

In [ ]:
from openai import OpenAI
import os

# Method 1: Set API key directly (for learning)
api_key = "your-api-key-here"  # Replace with your actual key

# Method 2: Use environment variable (recommended for production)
# api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

print("✓ OpenAI client initialized")

## Basic Chat Completion

In [ ]:
# Simple chat completion
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What is Python?"}
    ]
)

print("Response:")
print(response.choices[0].message.content)

## System Messages

System messages set the behavior/personality of the assistant.

In [ ]:
# Without system message
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Explain recursion"}
    ]
)
print("Without system message:")
print(response.choices[0].message.content[:200] + "...\n")

# With system message
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a computer science teacher explaining concepts to 10-year-olds. Use simple words and fun examples."},
        {"role": "user", "content": "Explain recursion"}
    ]
)
print("With system message (ELI10):")
print(response.choices[0].message.content[:200] + "...")

## Multi-Turn Conversations

In [ ]:
# Build conversation history
messages = [
    {"role": "system", "content": "You are a helpful Python tutor."},
    {"role": "user", "content": "What's a list comprehension?"}
]

# First response
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
assistant_message = response.choices[0].message.content
print("Assistant:", assistant_message[:150] + "...\n")

# Add to conversation history
messages.append({"role": "assistant", "content": assistant_message})
messages.append({"role": "user", "content": "Can you show me an example?"})

# Continue conversation
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)
print("Assistant:", response.choices[0].message.content)

## Response Properties

In [ ]:
# Inspect full response object
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello!"}]
)

print("Response structure:")
print(f"  ID: {response.id}")
print(f"  Model: {response.model}")
print(f"  Created: {response.created}")
print(f"  \nUsage:")
print(f"    Prompt tokens: {response.usage.prompt_tokens}")
print(f"    Completion tokens: {response.usage.completion_tokens}")
print(f"    Total tokens: {response.usage.total_tokens}")
print(f"  \nContent: {response.choices[0].message.content}")

## Parameters

Control generation with parameters.

In [ ]:
# Temperature: Controls randomness (0 = deterministic, 2 = creative)
prompt = "Write a tagline for a coffee shop"

print("Temperature = 0 (deterministic):")
for i in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    print(f"  {i+1}. {response.choices[0].message.content}")

print("\nTemperature = 1.5 (creative):")
for i in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=1.5
    )
    print(f"  {i+1}. {response.choices[0].message.content}")

In [ ]:
# max_tokens: Limit response length
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Write a long essay about Python"}],
    max_tokens=50  # Limit to ~50 tokens
)

print("Response (limited to 50 tokens):")
print(response.choices[0].message.content)
print(f"\nActual tokens used: {response.usage.completion_tokens}")

## Token Counting

In [ ]:
import tiktoken

# Count tokens in text
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

texts = [
    "Hello!",
    "This is a longer sentence with more words.",
    "tokenization converts text into numbers"
]

for text in texts:
    tokens = encoding.encode(text)
    print(f"Text: {text}")
    print(f"  Tokens: {tokens}")
    print(f"  Count: {len(tokens)}")
    print(f"  Decoded: {encoding.decode(tokens)}\n")

## Cost Estimation

In [ ]:
# Estimate cost before API call
def estimate_cost(messages, max_tokens=500, model="gpt-4o-mini"):
    """Estimate API call cost"""

    # Count input tokens
    encoding = tiktoken.encoding_for_model(model)
    input_tokens = 0
    for msg in messages:
        input_tokens += len(encoding.encode(msg["content"]))
    input_tokens += 3 * len(messages)  # Message overhead

    # Prices per 1M tokens
    prices = {
        "gpt-4o-mini": {"input": 0.15, "output": 0.60},
        "gpt-4o": {"input": 2.50, "output": 10.00}
    }

    input_cost = (input_tokens / 1_000_000) * prices[model]["input"]
    output_cost = (max_tokens / 1_000_000) * prices[model]["output"]

    return {
        "input_tokens": input_tokens,
        "max_output_tokens": max_tokens,
        "input_cost": input_cost,
        "max_output_cost": output_cost,
        "max_total_cost": input_cost + output_cost
    }

# Example
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a detailed explanation of machine learning."}
]

cost = estimate_cost(messages, max_tokens=500)
print("Cost Estimate:")
print(f"  Input tokens: {cost['input_tokens']}")
print(f"  Max output tokens: {cost['max_output_tokens']}")
print(f"  Input cost: ${cost['input_cost']:.6f}")
print(f"  Max output cost: ${cost['max_output_cost']:.6f}")
print(f"  Max total cost: ${cost['max_total_cost']:.6f}")

## Error Handling

In [ ]:
from openai import APIError, RateLimitError, APIConnectionError

def safe_api_call(messages):
    """API call with error handling"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

    except RateLimitError:
        return "Error: Rate limit exceeded. Please try again later."

    except APIConnectionError:
        return "Error: Connection failed. Check your internet."

    except APIError as e:
        return f"Error: API error - {str(e)}"

    except Exception as e:
        return f"Error: Unexpected error - {str(e)}"

# Test
result = safe_api_call([{"role": "user", "content": "Hello!"}])
print(result)

## Model Comparison

In [ ]:
# Compare different models
prompt = "Explain quantum computing in one sentence."

models = ["gpt-4o-mini", "gpt-4o"]

for model in models:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    content = response.choices[0].message.content
    tokens = response.usage.total_tokens

    print(f"Model: {model}")
    print(f"  Response: {content}")
    print(f"  Tokens: {tokens}\n")

## Summary

✅ Basic chat completions  
✅ System messages for behavior control  
✅ Multi-turn conversations  
✅ Temperature and max_tokens parameters  
✅ Token counting and cost estimation  
✅ Error handling  
✅ Model comparison

**Next:** Learn about structured outputs with Pydantic!